In [1]:
import numpy as np
import plotly.graph_objects as go
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from DEFAULTS import PARENT_PATH
import sim.ap_field_ar as ap
import sim.tele_geo_ar as tg
import sim.pan_mod_ar as pm

In [9]:
# default telescope geometry
tele_geo = tg.TelescopeGeometry()

# position of the receiver feed [mm]
P_rx = np.array([0, 209.920654, 0])

# arrays of pointing angles of rays
angle_size = 0.29
theta_a, theta_b, theta_N = -np.pi / 2 - angle_size, -np.pi / 2 + angle_size, 100
phi_a, phi_b, phi_N = np.pi / 2 - angle_size, np.pi / 2 + angle_size, 100
theta = np.linspace(theta_a, theta_b, theta_N)
phi = np.linspace(phi_a, phi_b, phi_N)

# create panel offsets randomly
save = 0 
error_rms = 0
adj_1_A = np.random.randn(1092) * error_rms
adj_2_A = np.random.randn(1092) * error_rms
panel_model2 = pm.panel_model_from_adjuster_offsets(
    2, adj_2_A, 1, save
)  # Panel Model on M2
panel_model1 = pm.panel_model_from_adjuster_offsets(
    1, adj_1_A, 1, save
)  # Panel Model on M1

# get parameters from telescope geometry
tg_th_1, tg_th2, tg_z_ap = tele_geo.th_1, tele_geo.th2, tele_geo.z_ap
tg_th_fwhp, tg_F_2 = tele_geo.th_fwhp, tele_geo.F_2

# ray tracing
rxmirror = ap.ray_mirror_pts(P_rx, tg_th2, tg_F_2, theta, phi)
apout = ap.aperature_fields_from_panel_model(panel_model1, panel_model2, \
                                    P_rx, tg_th_1, tg_th2, tg_z_ap, tg_th_fwhp, \
                                    theta, phi, rxmirror
                                    )

In [12]:
# plot the field on the aperture
k = 2*np.pi/tele_geo.lambda_ / 1000.0 # [1/mm]
aperture_radius = 3000 # [mm]
tg_rotc = np.array([tele_geo.x_rotc, tele_geo.y_rotc, tele_geo.z_rotc])*1e3 # center of rotation of the telescope [mm]
x_aprotc = apout[9] - tg_rotc[0]  # coordinates of pts on aperature relative to the rotation center [mm]
y_aprotc = apout[10] - tg_rotc[1]  # coordinates of pts on aperature relative to the rotation center [mm]
z_aprotc = apout[11] - tg_rotc[2]  # coordinates of pts on aperature relative to the rotation center [mm]
inap_bool = ((np.square(x_aprotc) + np.square(y_aprotc)) < np.square(aperture_radius))
valid_indices = np.where(inap_bool) #Ignore rays that missed the aperture plane

x_aprotc = x_aprotc[valid_indices]
y_aprotc = y_aprotc[valid_indices]
z_aprotc = z_aprotc[valid_indices]

intensity = apout[16][valid_indices]
pathl = apout[15][valid_indices]
ampl = np.sqrt(intensity)
field_ap = ampl * np.exp(1j * k * pathl)

plot_1 = go.Scatter(
                        x = x_aprotc, y = y_aprotc, 
                        mode = 'markers', 
                        marker=dict(symbol="circle", 
                                    colorscale='Twilight', color=np.angle(field_ap), 
                                    showscale=True, colorbar=dict(len=0.8, x=0.44)),
                        showlegend = False,
                        # hoverinfo='name', 
                        name="phase"
                        )
plot_2 = go.Scatter(
                        x = x_aprotc, y = y_aprotc, 
                        mode = 'markers', 
                        marker=dict(symbol="circle", 
                                    colorscale='Purpor', color=20*np.log10(np.abs(field_ap)/np.max(np.abs(field_ap))), 
                                    showscale=True, colorbar=dict(len=0.8, x=1)), 
                        showlegend = False,
                        # hoverinfo='name', 
                        name="power"
                        )


layout = go.Layout(title='Beam on Aperture Plane', autosize=True,
                width=1000, height=500, 
                margin=dict(l=50, r=50, b=65, t=90),
                xaxis1 = dict(title="x [mm]"),
                yaxis1 = dict(scaleanchor = 'x', title="y [mm]"),
                xaxis2 = dict(scaleanchor = "x", title="x [mm]"),
                yaxis2 = dict(scaleanchor = "y"),
                )

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, shared_yaxes=False, shared_xaxes=True, subplot_titles=["Phase [rad]", "Intensity [dB]"])
fig.add_trace(plot_1, row=1, col=1)
fig.add_trace(plot_2, row=1, col=2)
fig.update_layout(layout)
fig.show()

In [11]:
# here we show the simulated rays
tele_geo.draw_setup()
tele_geo.draw_rays(apout)
tele_geo.show_figure()
# tele_geo.figure.write_html('figure_rays.html', auto_open=True)